In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import Necessary Libraries

In [20]:
import cv2
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize , to_categorical
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Dropout ,Flatten , Dense , Activation

#Load the dataset

In [3]:
no_tumor_scans = os.listdir('/content/drive/MyDrive/Dataset/no') #store all the scans with no tumor in a array
tumor_scans = os.listdir('/content/drive/MyDrive/Dataset/yes')#store all the scans with tumor in a array


#Declare necessary varibles

In [4]:
dataset = []
label = []

#Data Preprocessing

In [5]:
for i , image_name in enumerate(no_tumor_scans):
  if(image_name.split('.')[1] == 'jpg'):
    image = cv2.imread('/content/drive/MyDrive/Dataset/no/'+image_name) #reading images
    image = Image.fromarray(image , 'RGB') #convert to RGB format for compatibility with opencv
    image = image.resize((64,64))
    dataset.append(np.array(image))
    label.append(0) #no brain_tumor has label of 0


for i , image_name in enumerate(tumor_scans):
  if(image_name.split('.')[1] == 'jpg'):
    image = cv2.imread('/content/drive/MyDrive/Dataset/yes/'+image_name) #reading images
    image = Image.fromarray(image , 'RGB') #convert to RGB format for compatibility with opencv
    image = image.resize((64,64))
    dataset.append(np.array(image))
    label.append(1) #no brain_tumor has label of 0

In [6]:
dataset = np.array(dataset)
label = np.array(label)

##Splitting dataset into train and test set

In [34]:
x_train , x_test , y_train , y_test = train_test_split(dataset , label , test_size = 0.2 , random_state = 0)


##Normalizing the data

In [35]:
x_train = normalize(x_train)
x_test = normalize(x_test)

##convert y_train and y_test to categorical

In [23]:
y_train = to_categorical(y_train , num_classes = 2)
y_test = to_categorical(y_test , num_classes = 2)

In [33]:
y_train.shape

(2400, 2, 2)

#Building the Model

In [39]:
model = Sequential()
model.add(Conv2D(32 , (3,3) , input_shape = (64,64,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32 , (3,3) , kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64 , (3,3) , kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

##Training the Model

In [40]:
model.fit(x_train , y_train , batch_size = 16 , verbose=1 , epochs = 10 , validation_data = (x_test , y_test) , shuffle = False)
model.save('BrainTumor10Epochs.h5')

Epoch 1/10
150/150 [==============================] - 3s 7ms/step - loss: 0.6268 - accuracy: 0.6125 - val_loss: 0.5780 - val_accuracy: 0.6383
Epoch 2/10
150/150 [==============================] - 1s 5ms/step - loss: 0.5482 - accuracy: 0.6921 - val_loss: 0.5461 - val_accuracy: 0.6650
Epoch 3/10
150/150 [==============================] - 1s 6ms/step - loss: 0.4999 - accuracy: 0.7146 - val_loss: 0.5215 - val_accuracy: 0.6733
Epoch 4/10
150/150 [==============================] - 1s 7ms/step - loss: 0.4632 - accuracy: 0.7400 - val_loss: 0.4959 - val_accuracy: 0.6900
Epoch 5/10
150/150 [==============================] - 1s 7ms/step - loss: 0.4260 - accuracy: 0.7504 - val_loss: 0.4808 - val_accuracy: 0.7000
Epoch 6/10
150/150 [==============================] - 1s 6ms/step - loss: 0.3860 - accuracy: 0.7717 - val_loss: 0.4258 - val_accuracy: 0.7350
Epoch 7/10
150/150 [==============================] - 1s 5ms/step - loss: 0.3431 - accuracy: 0.8108 - val_loss: 0.4060 - val_accuracy: 0.7517
Epoch 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
